# POS-tagging with Hidden Markov Models

### Reminder: POS-tagging

![pos](pos.jpg)

([SLP Ch.8](https://web.stanford.edu/~jurafsky/slp3/8.pdf))

### Supervised Machine Learning

**Train** a model using a large **corpus** of **annotated** text

### Baseline

_For each word $w$, choose the most frequent POS-tag in the training data_

$$ \operatorname{POS}(w) = \underset{l \in L}{\operatorname{argmax}} \mathbb{P}(l|w) $$

Where $L$ is the set of tags $l_1, l_2, \ldots$ and $$\mathbb{P}(l|w) \sim \frac{\#(w, l)}{\#w}$$

## Context matters too!

Remember **ngrams** ?

$$ l_i = \underset{l \in L}{\operatorname{argmax}} \mathbb{P}(l|(l_{i-1}, l_{i-2})) \sim \frac{\#(l_i, l_{i-1}, l_{i-2})}{\#(l_{i-1}, l_{i-2})}$$

We need to combine the two models of __likelihood__

## Hidden Markov Model (HMM)
<a id="hmm"></a>

We need the most likely tag sequence for a given sequence of words:

$$
\underset{l_i, l_j, \ldots, l_n \in L^N}{\operatorname{argmax}}\mathbb{P}(l_1, l_2, \ldots l_N \ | \ w_1, w_2 \ldots w_N)
$$

But we'll be looking for this:
$$
\underset{l_i, l_j, \ldots, l_n \in L^N}{\operatorname{argmax}}\mathbb{P}(w_1, w_2, \ldots w_N \ | \ l_1, l_2 \ldots l_N)
$$

Could we possibly approximate this as
$$
    \underset{l_i, l_j, \ldots, l_n \in L^N}{\operatorname{argmax}} \frac{\#((l_i, l_j, \ldots, l_n), (w_i, w_j, \ldots, w_n))}{\#(l_i, l_j, \ldots, l_n)}
$$
?

### Let's simplify a bit.

Suppose that:

- the current POS tag depends only on a fixed window of $n$ past tokens.

- word forms depend on their own POS tags only

### For $n=2$, let's optimize:

$$
\mathbb{P}(w_1, w_2, \ldots w_N \ | \ l_1, l_2 \ldots l_N) =
    \prod_{i=1}^N\mathbb{P}(l_i \ |\ l_{i-1})\cdot\mathbb{P}(w_i \ | \ l_i)
$$

* The terms $\mathbb{P}(l_i \ |\ l_{i-1})$ are called **transition probabilities**
* The terms $\mathbb{P}(w_i\ |\ l_i)$ are called **emission or observation probabilities**

#### These we can approximate based on the training data

$$\mathbb{P}(l_n\ |\ l_{n-1})\approx \frac{\#(l_{n-1},l_n)}{\#l_{n-1}}$$



$$ \mathbb{P}(w_i|l_i) \approx \frac{\#(w_i, l_i)}{\#l_i}$$

#### So how do we calculate the most likely tagset?

$$
\underset{l_i, l_j, \ldots, l_n \in L^N}{\operatorname{argmax}} \prod_{i=1}^N\mathbb{P}(l_i \ |\ l_{i-1})\cdot\mathbb{P}(w_i \ | \ l_i)
$$

# The Viterbi algorithm
<a id="viterbi"></a>


| |the|dog|saw|a|cat|
|-|---|---|---|-|---|
|**DET**| |   |   | |   |
|**NOUN**||   |   | |   |
|**VERB**||   |   | |   |


## The Viterbi algorithm
<a id="viterbi"></a>
(for $n=2$)

#### Input:
* the sentence $W = \{w_1, \ldots,w_N\}$
* the hidden state space $S=\{s_1, \ldots , s_{|L|}\}$ (the POS tags)
* the transition probabilities $T_{i,j} = \mathbb{P}(l_t=s_j|l_{t-1}=s_i)$
* the emission probabilities $E_{i,j} = \mathbb{P}(w_j|l_t=s_i)$

| |the|dog|saw|a|cat|
|-|---|---|---|-|---|
|**DET**| |   |   | |   |
|**NOUN**||   |   | |   |
|**VERB**||   |   | |   |


> For each word $w_i$ and for each possible label $l_j$<br/>
> $\quad$ calculate the probability of the most likely sequence ending in $(w_i, l_j)$<br/>
> $\quad$ $\pi[i,j] = \underset{k \in \{1, \ldots, |L|\}}{\operatorname{max}} \pi[k,j-1]\cdot T_{k,i} \cdot E_{i,j}$<br/>
> $\quad$ and make a note of the previous state for that sequence<br/>
>   $\quad$ $B[i,j]=\underset{k \in \{1, \ldots, |L|\}}{\operatorname{argmax}} \pi[k,j-1] \cdot T_{k,i} \cdot E_{i,j}$<br/>
> Finally, use the backpointers in $B$ to read out the most likely sequence of POS-tags

![viterbi](viterbi.jpg)

### Complexity

$\mathcal{O}(N \cdot |L|^2)$

### Notes

- How is the transition probability calculated for the first tag?

- How would the algorithm change for $n=3$ ? How does this change the time complexity?

- What could we do about words unseen in the training data? And unseen label sequences?

### HMMs work for all kinds of sequence labeling

#### Named Entity Recognition

![ner](ner_70.jpg)

| _American_ | _Airlines_ | , | _a_  | _unit_ | _of_ | _AMR_   | _Corp_  | _._     | _,_  | _immediately_ | _matched_ | _the_ | _move_ |
| :-------: | :-------: | :-:| :-: | :---: | :-: | :--:   | :---:  | :-:    | :-: | :-: | :-: | :-: | :-: |
| B-ORG    | I-ORG    | O | O  | O    | O  | B-ORG | I-ORG | I-ORG | O  | O  | O  | O  | O  |                                      

### NP-chunking

| _American_ | _Airlines_ | , | _a_  | _unit_ | _of_ | _AMR_   | _Corp_  | _._     | _,_  |
| :-------: | :-------: | :-:| :-: | :---: | :-: | :--:   | :---:  | :-:    | :-: | 
| B-NP    | I-NP    | O | B-NP  | I-NP  | I-NP  | I-NP | I-NP | I-NP | O  | 

| _immediately_ | _matched_ | _the_ | _move_ |
| :-: | :-: | :-: | :-: |
| O  | O  | B-NP  | I-NP  |

It is sometimes called __shallow parsing__.

# Questions?

You can implement your own HMM for POS-tagging with [this notebook](https://github.com/bmeaut/python_nlp_2018_spring/blob/master/homeworks/homework2/homework2.ipynb)